In [1]:
import os.path as op
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook', font_scale=1.2)
from pyplr import graphing, utils, preproc
from pyplr.plr import PLR
import numpy as np
import numpy as np
from scipy.signal import butter,filtfilt

## Utils

In [ ]:
def butter_lowpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y
def json_reader(path_to_file):
    with open(path_to_file) as f:
        data = f.read()
    #print("Data type before reconstruction : ", type(data))
    # reconstructing the data as a dictionary
    js = json.loads(data)
    #print("Data type after reconstruction : ", type(js))
    #print(js)
    return js
def json_writer(DATA,participant_no):
    # create json object from dictionary
    jsonobj = json.dumps(DATA)
    # open file for writing, "w" 
    save_file = str(participant_no)+".json"
    f = open(save_file,"w")
    # write json object to file
    f.write(jsonobj)
    f.close()
    return 0

In [2]:
def extract_timeseries(path):
        # Some useful constants
    SAMPLE_RATE = 200
    #DURATION = 1500
    #ONSET_IDX = 1
    # Columns to load
    use_cols = ['confidence',
                'method',
                'pupil_timestamp',
                'eye_id',
                'diameter_3d',
                'diameter']
    # Pupil Labs recording directories / exports
    subjects = {
        '001': [path, '000']
    }
    # Empty DataFrame to store processed PIPR data
    df = pd.DataFrame()

    # Loop over subjects
    for k in subjects.keys():
        # Get a handle on a subject
        rec = subjects[k][0]
        export = subjects[k][1]
        s = utils.new_subject(
            rec, export=export, out_dir_nm='pyplr_analysis')

        # Load pupil data
        samples = utils.load_pupil(
            s['data_dir'], eye_id='best', method='3d', cols=use_cols)

        # Pupil columns to analyse
        pupil_cols = ['diameter_3d', 'diameter']
        try:
            blinks  = utils.load_blinks(s['data_dir'])
        except:
            continue
    samples = preproc.mask_pupil_first_derivative(samples, threshold=4.0, mask_cols=pupil_cols)
    try:
        samples = preproc.mask_blinks(samples, blinks, mask_cols=['diameter_3d'])
    except:
        print('no blinks')

                    #Mask Pupil Confidence
    samples = preproc.mask_pupil_confidence(samples, threshold=0.75, mask_cols=pupil_cols)
    

                    #Low Pass Filter the Samples at a 1 Hz Cut off
                    #try:
                        #Interpolate Samples 
    samples = preproc.interpolate_pupil(samples, method='nearest', order=2,interp_cols=pupil_cols)
    samples = preproc.butterworth_series(samples, fields=pupil_cols, filt_order=3,cutoff_freq=1/(200/2))
    return samples

In [3]:
def pupil_extract(path,participant_no):
    count = 0
    experimentsrc_data =  json_reader('/Users/noelalben/github/pupil-pitch/Pupil-labs/compare_data.txt')
    participant_data= json_reader('/Users/noelalben/github/pupil-pitch/Pupil-labs/DATA.txt')
    frequencies = ['820','826','832','838','844','850','856','862','868','874','880']
    block_keys = ['block_1', 'block_2', 'block_3', 'block_4', 'block_5']
    user_dictionary = {}
    for i in frequencies:
        user_dictionary[i] = {}
        for j in block_keys:
            user_dictionary[i][j] = {}
            user_dictionary[i][j]['time_stamps']=[]
            user_dictionary[i][j]['event_stamps']=[]
            user_dictionary[i][j]['first_set']=[]
            user_dictionary[i][j]['second_set']=[]
            user_dictionary[i][j]['first_set_timestamps']=[]
            user_dictionary[i][j]['second_set_timestamps']=[]
            user_dictionary[i][j]['setup_to_next_trial'] = []
    trial_keys = ['trial_1', 'trial_2', 'trial_3', 'trial_4', 'trial_5']
    # Some useful constants
    SAMPLE_RATE = 200
    #DURATION = 1500
    #ONSET_IDX = 1
    # Columns to load
    use_cols = ['confidence',
            'method',
            'pupil_timestamp',
            'eye_id',
            'diameter_3d',
            'diameter']
    participant_info = experimentsrc_data[str(participant_no)]
    for block in range(1,6):
        block_folder = str(participant_no)+str(block)
        if(participant_no >= 10):
                block_folder = '0'+str(participant_no)+str(block)
                print(block_folder)
                flag = True
        else:
            flag = False
        freqs = []
        for trial_number in range(0,11):
            freq = participant_info[trial_keys[block-1]]['Stimuli'][trial_number][6:9]
            freqs.append(freq)
            print(freqs)
            user_block = user_dictionary[freq][block_keys[block-1]]
            
            #print(user_block)
            trial_folder = '00'+str(trial_number)
            if(trial_number == 10):
                trial_folder = '010'
            directory = path+'/00'+block_folder+'/'+trial_folder
            if(flag==True):
                directory = path+'/'+block_folder+'/'+trial_folder
            print(directory)
            # Pupil Labs recording directories / exports
            subjects = {
            '001': [directory, '000']
            }
            df = pd.DataFrame()

            # Loop over subjects
            for k in subjects.keys():
                # Get a handle on a subject
                rec = subjects[k][0]
                export = subjects[k][1]
                s = utils.new_subject(
                    rec, export=export, out_dir_nm='pyplr_analysis')

                # Load pupil data
                samples = utils.load_pupil(
                    s['data_dir'], eye_id='best', method='3d', cols=use_cols)
                blinks  = utils.load_blinks(s['data_dir'])

                # Pupil columns to analyse
                pupil_cols = ['diameter_3d', 'diameter']

                #First Derivative
                samples = preproc.mask_pupil_first_derivative(samples, threshold=3.0, mask_cols=pupil_cols)
                
                try:
                    #Mask Blinks 
                    samples = preproc.mask_blinks(samples, blinks, mask_cols=['diameter_3d'])

                    #Mask Pupil Confidence
                    samples = preproc.mask_pupil_confidence(samples, threshold=0.75, mask_cols=pupil_cols)


                    #Low Pass Filter the Samples at a 1 Hz Cut off
                    #try:
                        #Interpolate Samples 

                    samples = preproc.interpolate_pupil(samples, method='nearest', order=2,interp_cols=pupil_cols)
                    samplestmp = samples
                    samples1 = preproc.butterworth_series(samplestmp, fields=pupil_cols, filt_order=3,cutoff_freq=1/(200/2))
                    if (samples1['diameter_3d'].isnull().all() == True):
                        samples = samples
                    else:
                        samples = samples1
                   # except samples['diameter_3d'].isnull().all() == True:
                except Exception:
                                        #Mask Pupil Confidence
                    samples = preproc.mask_pupil_confidence(samples, threshold=0.75, mask_cols=pupil_cols)


                    #Low Pass Filter the Samples at a 1 Hz Cut off
                    #try:
                        #Interpolate Samples 

                    samples = preproc.interpolate_pupil(samples, method='nearest', order=2,interp_cols=pupil_cols)
                    samplestmp = samples
                    samples1 = preproc.butterworth_series(samplestmp, fields=pupil_cols, filt_order=3,cutoff_freq=1/(200/2))
                    if (samples1['diameter_3d'].isnull().all() == True):
                        samples = samples
                    else:
                        samples = samples1
                   # except samples['diameter_3d'].isnull().all() == True:
                    
                    
                    

                events = utils.load_annotations(s['data_dir'])
                event_qeury = events.axes[0].tolist()
                event_qeury = [round(float(i), 2) for i in event_qeury]
                qeury = samples.axes[0].tolist()
                query = [round(float(i), 2) for i in qeury ]
                user_block['time_stamps'] = query
                user_block['event_stamps'] = event_qeury
                
                interpolation = np.size(np.where(samples['interpolated']==1)[0])
                all_values =  np.size(samples['interpolated'])
                percs = interpolation/all_values
                if(percs>0.8):
                    user_block['first_set'] = 'Reject'
                    user_block['second_set'] = 'Reject'
                    count = count+1
                    print('------here--------')
                    continue

                #Plotting the samples for participant
                #First two Events 
                try:
                    samples_extract = samples.iloc[query.index(event_qeury[0]):query.index(event_qeury[1])+600,:].iloc[::50,:].axes[0].tolist()
                    user_block['first_set_timestamps'] = samples_extract
                    user_block['untilevent0'] =samples.iloc[0:query.index(event_qeury[0]),:]['diameter_3d'].tolist()
                    user_block['first_set'] = samples.iloc[query.index(event_qeury[0]):query.index(event_qeury[1])+600,:].iloc[::50,:]['diameter_3d'].tolist()
                    if trial_number>0:
                        print(trial_number)
                        user_block_prev = user_dictionary[freqs[trial_number-1]][block_keys[block-1]]
                        setup = user_block_prev['setup_to_next_trial']
                        if(user_block_prev['first_set'] != 'Reject'):
                            setup = setup[len(setup) - 1::-1][0:12]
                            setup = setup[len(setup) - 1::-1]
                            user_block['first_set'][:0] = setup[0:12]

               # Second set of Events
                    samples_extract2 = samples.iloc[query.index(event_qeury[2]):query.index(event_qeury[4])+800,:].iloc[::50,:].axes[0].tolist()
                    user_block['second_set_timestamps']=samples_extract2
                    
                    user_block['second_set'] = samples.iloc[query.index(event_qeury[2]):query.index(event_qeury[4])+800,:].iloc[::50,:]['diameter_3d'].tolist()
             # Anticipation of new block
                    samples_extract3 = samples.iloc[query.index(event_qeury[4])+200:,:].iloc[::50,:]['diameter_3d'].tolist()
                    user_block['setup_to_next_trial'] = samples_extract3
                    
                except Exception:
                    continue
                
    json_writer(user_dictionary,participant_no)
    
    return user_dictionary

In [ ]:
def Answer_check(Comparison_Hz, Block_number,participantsrc_data,participant_respond_data):
    Trials = ['trial_1','trial_2','trial_3','trial_4','trial_5']
    comparison_flag = 'rshift'
    if (Comparison_Hz<850):
        comparison_flag = 'lshift'
    
    participant_data = participantsrc_data[Trials[Block_number-1]]
    participant_data_stimuli = np.asarray(participant_data['Stimuli'])
    checker = 'Audio/'+str(Comparison_Hz)+'.0.wav'
    print(checker)
    find = np.where(checker == participant_data_stimuli)[0]
    participant_response_data = participant_respond_data[Trials[Block_number-1]]
    print(participant_response_data)
    print(participant_data['Answers'])
    print(find)
    if(participant_response_data[0][find[0]] == participant_data['Answers'][find[0]]):
        Answer = 'Correct'
    elif (Comparison_Hz == 850):
        Answer = 'Correct'
    else:
        Answer = 'Incorrect'
    return Answer

In [ ]:
path = '/Users/noelalben/Desktop/Pupil-PitchData/EyeRecording'
dics = []
for i in range(3,15):
    dics.append(pupil_extract(path,i))

In [ ]:
path = '/Users/noelalben/Desktop/Pupil-PitchData/EyeRecording'
dic_5 = pupil_extract(path,6)

In [ ]:
df_list = [pd.DataFrame(columns=['Participant','Comparison Hertz','3s-pre-ready','2s-pre-ready','1s-pre-ready','0th-time','at-ready','1s-post-ready','2s-post-ready','at-standard','1s-post-standard','2s-post-standard','3s-post-standard','at-comparison','1s-post-comparison','2s-post-comp','3s-post-comp','at-response','1s-post-response','2s-post-response','3s-post-response','4s-post-response','Response (Correct/Incorrect)','Baseline'],index = indexes) for x in range(11)]

In [ ]:
k = 0
for df_iter in df_list:
    #df_iter = pd.DataFrame(columns=['Participant','Comparison Hertz','3s-pre-ready','2s-pre-ready','1s-pre-ready','at-ready','1s-post-ready','2s-post-ready','at-standard','1s-post-standard','2s-post-standard','3s-post-standard','at-comparison','1s-post-comparison','2s-post-comp','3s-post-comp','at-response','1s-post-response','2s-post-response','3s-post-response','4s-post-response','Response (Correct/Incorrect)'],index = indexes)
    path = '/Users/noelalben/github/pupil-pitch/'
    freq_in_check =int(frequencies[k])
    k = k+1
    for i in range(3,15):
        read_path = path + str(i)+'.json'
        user_dictionary = json_reader(read_path)
        for j in range(1,6):
            user_block_first_set = user_dictionary[str(freq_in_check)][block_keys[j-1]]['first_set']
            user_block_second_set = user_dictionary[str(freq_in_check)][block_keys[j-1]]['second_set']
            Answer = Answer_check(freq_in_check, j-1,experimentsrc_data[str(i)],participant_data[str(i)][str(i)]['Key_Responds'])
            try:
                user_block_begining = np.mean(user_dictionary[str(freq_in_check)][block_keys[j-1]]['untilevent0'][0:-10])
            except:
                user_block_begining = []
            participant_block = str(i)+'_'+str(j)
            print(participant_block)
            try:
                if(len(user_block_first_set)<20):
                    df_iter.loc[participant_block] = pd.Series({'Participant':i,'Comparison Hertz':freq_in_check,'0th-time': user_block_begining, 'at-ready':user_block_first_set[2],'1s-post-ready':user_block_first_set[6],'2s-post-ready':user_block_first_set[10],'at-standard':user_block_second_set[0],'1s-post-standard':user_block_second_set[4],'2s-post-standard':user_block_second_set[8],'3s-post-standard':user_block_second_set[12],'at-comparison':user_block_second_set[16],'1s-post-comparison':user_block_second_set[20],'2s-post-comp':user_block_second_set[24],'3s-post-comp':user_block_second_set[28],'at-response':user_block_second_set[32],'1s-post-response':user_block_second_set[36],'2s-post-response':user_block_second_set[40],'3s-post-response':user_block_second_set[44],'4s-post-response':user_block_second_set[47],'Response (Correct/Incorrect)':Answer,'Baseline':user_block_begining})
                else:   
                    df_iter.loc[participant_block] = pd.Series({'Participant':i,'Comparison Hertz':freq_in_check,'3s-pre-ready':user_block_first_set[0],'2s-pre-ready':user_block_first_set[4],'1s-pre-ready':user_block_first_set[8],'at-ready':user_block_first_set[2+12],'1s-post-ready':user_block_first_set[6+12],'2s-post-ready':user_block_first_set[10+12],'at-standard':user_block_second_set[0],'1s-post-standard':user_block_second_set[4],'2s-post-standard':user_block_second_set[8],'3s-post-standard':user_block_second_set[12],'at-comparison':user_block_second_set[16],'1s-post-comparison':user_block_second_set[20],'2s-post-comp':user_block_second_set[24],'3s-post-comp':user_block_second_set[28],'at-response':user_block_second_set[32],'1s-post-response':user_block_second_set[36],'2s-post-response':user_block_second_set[40],'3s-post-response':user_block_second_set[44],'4s-post-response':user_block_second_set[47],'Response (Correct/Incorrect)':Answer,'Baseline':np.mean(user_block_first_set[0:8:4])})
            except:
                df_iter.loc[participant_block] = pd.Series({'Participant':i, 'Comparison Hertz':freq_in_check, '3s-pre-ready': 'REJECT'})
                continue

In [ ]:
#Calculating Baseline
frequencies = ['820','826','832','838','844','850','856','862','868','874','880']
path = '/Users/noelalben/github/pupil-pitch/'
c= 0
baselines ={}
for k in range(3,15):
    baselines[k] = {}
for k in range(3,15):
    read_path = path + str(k)+'.json'
    user_dictionary = json_reader(read_path)
    for j in range(1,6):
        baseline = []
        for i in range(0,11):
            freq_in_check =int(frequencies[i])
            user_block_first_set = user_dictionary[str(freq_in_check)][block_keys[j-1]]['first_set']
            if(len(user_block_first_set)>20):
                baseline.append(user_block_first_set[0:12])
            else:
                print('here')
                c = c+1
                print(c)
        flat_list = [item for sublist in baseline for item in sublist]
        
        baselines[k][j] = sum(flat_list)/len(flat_list)

## Variation

In [ ]:
#df_s = [df_820,df_826,df_832,df_838,df_844,df_850,df_856,df_862,df_868,df_874,df_880]
frequencies = ['820','826','832','838','844','850','856','862','868','874','880']
indexes = ['4_1','4_2','4_3','5_1','5_2','5_3','6_1','6_2','6_3','7_1','7_2','7_3','8_1','8_2','8_3','9_1','9_2','9_3','11_1','11_2','11_3','12_1','12_2','12_3','13_1','13_2','13_3','14_1','14_2','14_3','16_1','16_2','16_3','17_1','17_2','17_3']
frequencies = ['820','826','832','838','844','850','856','862','868','874','880']

indexes = ['4_1','4_2','4_3','5_1','5_2','5_3','6_1','6_2','6_3','7_1','7_2','7_3','8_1','8_2','8_3','9_1','9_2','9_3','11_1','11_2','11_3','12_1','12_2','12_3','13_1','13_2','13_3','14_1','14_2','14_3']
df_880= pd.DataFrame(columns=['Participant','Comparison Hertz','3s-pre-ready','2s-pre-ready','1s-pre-ready','at-ready','1s-post-ready','2s-post-ready','at-standard','1s-post-standard','2s-post-standard','3s-post-standard','at-comparison','1s-post-comparison','2s-post-comp','3s-post-comp','at-response','1s-post-response','2s-post-response','3s-post-response','4s-post-response','Response (Correct/Incorrect)'],index = indexes)


In [ ]:
def Answer_check(Comparison_Hz, Block_number,participantsrc_data,participant_respond_data):
    Trials = ['trial_1','trial_2','trial_3']
    comparison_flag = 'rshift'
    if (Comparison_Hz<850):
        comparison_flag = 'lshift'
    
    participant_data = participantsrc_data[Trials[Block_number-1]]
    participant_data_stimuli = np.asarray(participant_data['Stimuli'])
    checker = 'Audio/'+str(Comparison_Hz)+'.0.wav'
    print(checker)
    find = np.where(checker == participant_data_stimuli)[0]
    participant_response_data = participant_respond_data[Trials[Block_number-1]]
    print(participant_response_data)
    print(participant_data['Answers'])
    print(find)
    if(participant_response_data[0][find[0]] == participant_data['Answers'][find[0]]):
        Answer = 'Correct'
    elif (Comparison_Hz == 850):
        Answer = 'Correct'
    else:
        Answer = 'Incorrect'
    return Answer